In [1]:
'''
Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe
10/20/2021

clause representation: 

  [<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

  [ 'p', 'q', 'r' ]

  Note: 'p', 'q', 'r', 's' are propositions : CASE SENSITIVE!

Example clause: 

  [['p', 'q'] ['r']] : This is clause P \/ Q \/ ~R 

'''

DEBUG=True

#---------------------------------------------------------------------
def rm_item(lst, item):  
#---------------------------------------------------------------------
  '''
  function: rm_pred()
  
  remove item from list
  - return [] if empty, not None
  '''
  lst.remove(item)

  if lst == None:
    return []
  else: 
    return lst


#---------------------------------------------------------------------
def mk_unique(clause):
#---------------------------------------------------------------------
  '''
  remove redundant propositions in the clause
  '''
  pos = clause[0]
  neg = clause[1]

  pos = list(set(pos))
  neg = list(set(neg))

  return [pos, neg]

#---------------------------------------------------------------------
def resolve(clause1, clause2):
#---------------------------------------------------------------------
  ''' 
  function: resolve() 

  [['p', 'q'] ['r']] : clause P \/ Q \/ ~R 

  resolve clauses c1 and c2 
  - if resolvable returns resolvent
  - if not resolvable returns False
  - returns empty clause [[], []] when Falsified

  '''

  # remove any redundant propositions in the clauses
  c1 = mk_unique(clause1)
  c2 = mk_unique(clause2)

  count = 0

  # print clauses
  if DEBUG:
    print("resolving: "+str(c1)+" and "+str(c2))

  # c1 pos vs. c2 neg
  for p1 in c1[0]:
    for p2 in c2[1]:
      if p1==p2:
        c1 = [rm_item(c1[0],p1), c1[1]]
        c2 = [c2[0], rm_item(c2[1],p2)]
        count = count+1

  # c2 pos vs. c1 neg
  for p1 in c1[1]:
    for p2 in c2[0]:
      if p1==p2:
        c1 = [c1[0], rm_item(c1[1],p1)]
        c2 = [rm_item(c2[0],p2), c2[1]]
        count = count+1

  # check for multiple matches or no match and abort if so.
  if count>1 or count==0:
    return False
  
  # compute resolvent
  c1[0].extend(c2[0])
  c1[1].extend(c2[1])

  # make unique and return
 
  return mk_unique(c1)



#---------------------------------------------------------------------
def null_p(clause):
#---------------------------------------------------------------------
  '''
  function null_p()

  check if "False" is derived 
  - returns True if empty clause
  - returns False if non-empty clause
  '''

  if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
    return True
  else:
    return False



#---------------------------------------------------------------------
def print_clause(clause):
#---------------------------------------------------------------------
  '''
  function print_clause

  See function resolve() for clause representation.

  '''

  s = ""
  for pos in clause[0]:
    s = s+pos+"\/"
  
  for neg in clause[1]:
    s = s+"~"+neg+"\/"
 
  if len(s)==0:
    print("NULL")
  else:
    n=len(s)
    print(s[0:(n-2)])

#------------------------------------------------------
# test resolve() function
#------------------------------------------------------

print("\ntest1")
test1 = resolve([['p', 'q'], ['r']], [[], ['p']])
print("resolvent = "+str(test1))
print("null? "+str(null_p(test1)))

print("\ntest2")
test2 = resolve([['p'], []], [[], ['p']])
print("resolvent = "+str(test2))
print("null? "+str(null_p(test2)))

print("\ntest3")
test3 = resolve([['p', 'q'], ['r']], [['r'], ['p']])
print("resolvent = "+str(test3))
print("null? "+str(null_p(test3)))

print("\ntest4")
test4 = resolve([['p', 'q'], ['r']], [['r'], []])
print("resolvent = "+str(test4))
print("null? "+str(null_p(test4)))


print("\ntest5")
test5 = resolve([['p', 'q'], ['r']], [['p', 'r'], []])
print("resolvent = "+str(test5))
print("null? "+str(null_p(test5)))


# edge case not being handled correctly: p \/ q \/ ~r \/ ~r  vs. p \/ r should give p \/ q, not p \/ q \/ ~r 
# - now this is fixed, with the use of mk_unique()
print("\ntest6")
test6 = resolve([['p', 'q'], ['r', 'r']], [['p', 'r'], []])
print("resolvent = "+str(test6))
print("null? "+str(null_p(test6)))



test1
resolving: [['q', 'p'], ['r']] and [[], ['p']]
resolvent = [['q'], ['r']]
null? False

test2
resolving: [['p'], []] and [[], ['p']]
resolvent = [[], []]
null? True

test3
resolving: [['q', 'p'], ['r']] and [['r'], ['p']]
resolvent = False
null? False

test4
resolving: [['q', 'p'], ['r']] and [['r'], []]
resolvent = [['q', 'p'], []]
null? False

test5
resolving: [['q', 'p'], ['r']] and [['r', 'p'], []]
resolvent = [['q', 'p'], []]
null? False

test6
resolving: [['q', 'p'], ['r']] and [['r', 'p'], []]
resolvent = [['q', 'p'], []]
null? False


In [2]:
'''
Using the resolve() function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

theorem representation:

[ [<index> <clause1>], [<index> <clause2>], ... ]

index : integer 

clause representation:

[<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

[ 'p', 'q', 'r' ]

'''

# define your theorem, as a set of indexed clauses

theorem = [    # pos-list      # neg-list 
           [0, [['p','q'],    ['r']]],       # clause 0     P \/ Q \/ ~R
           [1, [['s'],        ['q']]],       # clause 1     S \/ ~Q 
           [2, [['t'],        ['s']]],       # clause 2     T \/ ~S 
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R   
           [5, [[],           ['t']]]        # clause 5     ~T  
          ]

theorem2 = [ ]  # enter the example theorm in the homework 

# clause 0: I\/~M
# clause 1: M\/~I
# clause 2: M\/L
# clause 3: H\/~I
# clause 4: H\/~L
# clause 5: G\/~H

unicorn_theorm = [  # pos-list      # neg-list 
           [0,      [['I',],    ['M']]],      
           [1,      [['M'],        ['I']]],       
           [2,      [['M','L'],        []]],      
           [3,      [['H'],        ['I']]],       
           [4,      [['H'],        ['L']]],            
           [5,      [['G'],           ['H']]]         
          ]

theorem3 = [   # pos-list     # neg-list    
           [0, [['A'],        []]],        # clause 0     A
           [1, [['B'],        []]],        # clause 1     B
           [2, [['D'],         []]],       # clause 2    D
           [3, [['C'],        ['A','B']]],   # clause 3     ¬A ∨ ¬B ∨ C   
           [4, [['E'],        ['C','D']]],   # clause 4     ¬C ∨ ¬D ∨ E   
           [5, [[],           ['E']]]        # clause 5     ~E - negated conclusion starts here (goal clause index = 5)          

 ]  # enter your own example for testing 


#------------------------------------------------------
# prover() function : you must implement this function 
#---------------------------------------------------------------------
def prover(thm, goal):
#---------------------------------------------------------------------
  '''
  function prover: implement this

  two-pointer method, with set of support
  - arguments: 
    thm : theorem
    goal : integer index (clause number where the negated conclusion starts)
  - show resolution steps
  - if null_p checks, return True (theorem proven)
  - otherwise return False
  '''

  print('\nprover():\n\nTheorem:')

  # Initialize variables

  inner=0
  outer=goal

  intial_num_clause=len(thm)
  proved=False
  
  while outer<len(thm):

    
    resolvent=resolve(thm[inner][1], thm[outer][1])
    

    if resolvent is not False:
      
      print("resolvent:",end="")
      print_clause(resolvent)

      if null_p(resolvent):
        
        proved=True
        break

      else:
        flag=True
        for _,clause in thm:
          if mk_unique(clause)==resolvent:
            flag=False
            break
        if flag:
          cnt=len(thm)
          thm.append([cnt,resolvent])

      
    inner+=1
    if inner==outer:
      inner=0
      outer+=1


  print('Goal clause index=',end='')
  print(goal)

  
  for i,clause in enumerate(thm):
    if i==intial_num_clause:
      print(f'The number of new clauses is {len(thm)-intial_num_clause}')
      print('New clauses')
    print("clause "+str(clause[0])+": ",end="")
    print_clause(clause[1])

  # print()



  
  return proved

#------------------------------------------------------
# test prover() function
#------------------------------------------------------

# Test prover on example theorem


print('Test the prover() function on the default theorem in the skeleton code')

prover(theorem, 3)





Test the prover() function on the default theorem in the skeleton code

prover():

Theorem:
resolving: [['q', 'p'], ['r']] and [[], ['p']]
resolvent:q\/~r
resolving: [['s'], ['q']] and [[], ['p']]
resolving: [['t'], ['s']] and [[], ['p']]
resolving: [['q', 'p'], ['r']] and [['r'], []]
resolvent:q\/p
resolving: [['s'], ['q']] and [['r'], []]
resolving: [['t'], ['s']] and [['r'], []]
resolving: [[], ['p']] and [['r'], []]
resolving: [['q', 'p'], ['r']] and [[], ['t']]
resolving: [['s'], ['q']] and [[], ['t']]
resolving: [['t'], ['s']] and [[], ['t']]
resolvent:~s
resolving: [[], ['p']] and [[], ['t']]
resolving: [['r'], []] and [[], ['t']]
resolving: [['q', 'p'], ['r']] and [['q'], ['r']]
resolving: [['s'], ['q']] and [['q'], ['r']]
resolvent:s\/~r
resolving: [['t'], ['s']] and [['q'], ['r']]
resolving: [[], ['p']] and [['q'], ['r']]
resolving: [['r'], []] and [['q'], ['r']]
resolvent:q
resolving: [[], ['t']] and [['q'], ['r']]
resolving: [['q', 'p'], ['r']] and [['q', 'p'], []]
resolvin

True

# Unicorn theorm

In [3]:
# unicorn theorm

print('Test the prover() function on the the Unicorn theorem')

negated_conclusions=[
                    [[],['M']],
                    [[],['G']],
                    [[],['H']]
                    ]

statements=['The unicorn is mythical','The unicorn is magical','The unicorn is horned ']
n=len(unicorn_theorm)
for statement,negated_conclusion in zip(statements,negated_conclusions):

  print('***********************')

  print(f'Conclusion:{statement}')

  unicorn_theorm_copy=unicorn_theorm.copy()
  unicorn_theorm_copy.append([n,negated_conclusion])
  if prover(unicorn_theorm_copy,n):
    print('Successfully Proved')
  else:
    print('Failed to Prove')

  print('-----------------------')

Test the prover() function on the the Unicorn theorem
***********************
Conclusion:The unicorn is mythical

prover():

Theorem:
resolving: [['I'], ['M']] and [[], ['M']]
resolving: [['M'], ['I']] and [[], ['M']]
resolvent:~I
resolving: [['L', 'M'], []] and [[], ['M']]
resolvent:L
resolving: [['H'], ['I']] and [[], ['M']]
resolving: [['H'], ['L']] and [[], ['M']]
resolving: [['G'], ['H']] and [[], ['M']]
resolving: [['I'], ['M']] and [[], ['I']]
resolvent:~M
resolving: [['M'], ['I']] and [[], ['I']]
resolving: [['L', 'M'], []] and [[], ['I']]
resolving: [['H'], ['I']] and [[], ['I']]
resolving: [['H'], ['L']] and [[], ['I']]
resolving: [['G'], ['H']] and [[], ['I']]
resolving: [[], ['M']] and [[], ['I']]
resolving: [['I'], ['M']] and [['L'], []]
resolving: [['M'], ['I']] and [['L'], []]
resolving: [['L', 'M'], []] and [['L'], []]
resolving: [['H'], ['I']] and [['L'], []]
resolving: [['H'], ['L']] and [['L'], []]
resolvent:H
resolving: [['G'], ['H']] and [['L'], []]
resolving: [[],

# Testing for example

In [4]:
print('Enter your own example for testing ')

prover(theorem3,5)

Enter your own example for testing 

prover():

Theorem:
resolving: [['A'], []] and [[], ['E']]
resolving: [['B'], []] and [[], ['E']]
resolving: [['D'], []] and [[], ['E']]
resolving: [['C'], ['B', 'A']] and [[], ['E']]
resolving: [['E'], ['D', 'C']] and [[], ['E']]
resolvent:~D\/~C
resolving: [['A'], []] and [[], ['D', 'C']]
resolving: [['B'], []] and [[], ['D', 'C']]
resolving: [['D'], []] and [[], ['D', 'C']]
resolvent:~C
resolving: [['C'], ['B', 'A']] and [[], ['D', 'C']]
resolvent:~B\/~A\/~D
resolving: [['E'], ['D', 'C']] and [[], ['D', 'C']]
resolving: [[], ['E']] and [[], ['D', 'C']]
resolving: [['A'], []] and [[], ['C']]
resolving: [['B'], []] and [[], ['C']]
resolving: [['D'], []] and [[], ['C']]
resolving: [['C'], ['B', 'A']] and [[], ['C']]
resolvent:~B\/~A
resolving: [['E'], ['D', 'C']] and [[], ['C']]
resolving: [[], ['E']] and [[], ['C']]
resolving: [[], ['D', 'C']] and [[], ['C']]
resolving: [['A'], []] and [[], ['B', 'A', 'D']]
resolvent:~B\/~D
resolving: [['B'], []] a

True